In [ ]:
from rdkit import Chem
import pandas as pd
import sys
sys.path.append('..')
from utils import *
import glob
import os

In [ ]:
metanetx_df = pd.read_csv('metanetx/10May2023_metanetx_chem_prop.tsv', sep='\t', comment='#').dropna(subset=['SMILES'])
mnx_to_smiles = pd.Series(metanetx_df['SMILES'].values, index=metanetx_df['ID'].values).to_dict()

In [ ]:
# organism_name = 'p_aeruginosa'
# organism_name = 'e_coli'
# organism_name = 'p_putida'
# organism_name = 'p_solanacearum'
# organism_name = 'k_pneumoniae'
organism_name = 'z_mays'
CPDS_FILES = glob.glob(os.path.join(organism_name,"*chemicals.tsv"))
SAVE_NAME = "{}/{}_metabolites_from_metanetx_pathways.csv".format(organism_name, organism_name)

print (CPDS_FILES)

In [ ]:
cpds_df = pd.concat([pd.read_csv(file, sep='\t', comment = '#', header=None) for file in CPDS_FILES])
cpds_df

In [ ]:
cpds_df.columns = ['ID','name','source','formula','mass','charge','xrefs']
cpds_df['smiles'] = [standardize_smiles(mnx_to_smiles[i]) if i in mnx_to_smiles.keys() else None for i in cpds_df['ID']]
cpds_df = cpds_df.dropna(subset='smiles')
cpds_df = cpds_df[cpds_df['smiles'].map(lambda x : '*' not in x)]
cpds_df = cpds_df.drop_duplicates(subset=['smiles'])

cpds_df

In [ ]:
## Uncommment to save

pd.DataFrame({'name':cpds_df['name'].tolist(), 'smiles':cpds_df['smiles'].tolist()}).to_csv(SAVE_NAME, index=False, sep='\t')
print ("DATA SAVED TO", SAVE_NAME)

In [ ]:
metanetx_df

In [ ]:
metanetx_df['std_smiles'] = [standardize_smiles(s) for s in metanetx_df['SMILES']]

In [ ]:
deduped = metanetx_df.drop_duplicates(subset='std_smiles')

In [ ]:
deduped['mol'] = [Chem.MolFromSmiles(s) for s in deduped['std_smiles']]

In [ ]:
deduped[deduped['std_smiles'].map(lambda x : '*' not in x)]